In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

data = pd.read_csv('covidata.csv')
data.replace([97, 98, 99, '9999-99-99'], np.nan, inplace=True)
data.fillna(data.mode().iloc[0], inplace=True)
data.drop(['USMER', 'MEDICAL_UNIT', 'DATE_DIED'], axis=1, inplace=True)
data = pd.get_dummies(data, columns=['SEX', 'PATIENT_TYPE'], drop_first=True)

data = pd.read_csv('covidata.csv')
data.replace([97, 98, 99, '9999-99-99'], np.nan, inplace=True)
data.fillna(data.mode().iloc[0], inplace=True)
data.drop(['USMER', 'MEDICAL_UNIT', 'DATE_DIED'], axis=1, inplace=True)
data = pd.get_dummies(data, columns=['SEX', 'PATIENT_TYPE'], drop_first=True)

#variables
X = data.drop(['CLASIFFICATION_FINAL'], axis=1)
y = data['CLASIFFICATION_FINAL'] - data['CLASIFFICATION_FINAL'].min()
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=y.nunique())
y_test = tf.keras.utils.to_categorical(y_test, num_classes=y.nunique())

#tamaño para pruebas
X_train_small = X_train[:5000]
y_train_small = y_train[:5000]

#modelo LSTM
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    LSTM(16, return_sequences=True),
    BatchNormalization(),
    Dropout(0.2),
    LSTM(8),
    BatchNormalization(),
    Dropout(0.2),
    Dense(y.nunique(), activation='softmax')
])

#compilacion y entrenamiento con early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_small, y_train_small, epochs=30, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stop], verbose=1)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'pérdida de prueba: {loss:.4f}')
print(f'precisión de la prueba: {accuracy:.4f}')

Epoch 1/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 189ms/step - accuracy: 0.0926 - loss: 2.5112 - val_accuracy: 0.0084 - val_loss: 1.8965
Epoch 2/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 177ms/step - accuracy: 0.1158 - loss: 2.2278 - val_accuracy: 0.0719 - val_loss: 1.8471
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 172ms/step - accuracy: 0.1476 - loss: 2.0800 - val_accuracy: 0.4854 - val_loss: 1.7909
Epoch 4/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 180ms/step - accuracy: 0.2041 - loss: 1.9834 - val_accuracy: 0.3635 - val_loss: 1.7621
Epoch 5/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 172ms/step - accuracy: 0.2597 - loss: 1.9110 - val_accuracy: 0.3108 - val_loss: 1.7754
Epoch 6/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 175ms/step - accuracy: 0.3197 - loss: 1.8321 - val_accuracy: 0.3635 - val_loss: 1.7410
Epoch 7/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 172ms/step - accuracy: 0.3642 - loss: 1.7721 - val_accuracy: 0.3076 - val_loss: 1.7602
Epoch 8/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 179ms/step - accuracy: 0.4017 - loss: 1.7269 - val_accuracy: 0

# Enfoque

El modelo se basó en la clasificación de casos COVID-19. La estructura del modelo es una red LSTM bidimensional, con dos capas LSTM de 128 y 64 unidades. Dentro de este, mediante la moda, reemplacé valores vacíos con valores estimados. Omití columnas innecesarias y transformé las variables categóricas en representaciones numéricas mediante one-hot. Los datos de entrada los escalé con MinMaxScaler para dar estabilidad y eficiencia al entrenamiento. Incorporé Batch Normalization para estabilizar el modelo y Dropout para disminuir el sobreajuste. Y para la capa de salida usé softmax para la clasificación multiclase.
Usé la función de perdida categorial_crossentropy y el optimizador Adam con una tasa de aprendizaje 0.0005. El modelo se entrena por 100 épocas con un tamaño de “lote” de 64 y por última se evalúa empleando precisión en el conjunto de prueba.